# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [38]:
# Dependencies and Setup.
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Importing our API key.
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [103]:
# Importing our CSV file from Part 1.
vacation_weather_df = pd.read_csv("/Users/azpunit/Desktop/python-api-challenge/WeatherPy/final_output_data/City_Data.csv")

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [40]:
# Configuring our gmap.
gmaps.configure(api_key=g_key)

In [104]:
# Plotting our heatmap.
lat_lng_loc = vacation_weather_df[["Lat", "Lng"]]
humidity = vacation_weather_df["Humidity"]

layout = {"width": "900px",
          "height": "600px",
          "border": "1px solid black",
          "padding": "1px",
          "margin": "0 auto 0 auto"
          }

my_gmap = gmaps.figure(layout=figure_layout, zoom_level=2, center=(15,25))
heat_structure = gmaps.heatmap_layer(lat_lng_loc, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3.5)

my_gmap.add_layer(heat_structure)
my_gmap

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [114]:
# Creating a new data frame filling the weather criteria given to us.
columns = ["City_ID", "City", "Cloudiness", "Country", "Date", "Humidity", "Lat", "Lng", "Max Temp", "Wind Speed"]

intermediary_vacation_weather_df = vacation_weather_df.loc[(vacation_weather_df["Max Temp"] < 80) &
                                                           (vacation_weather_df["Max Temp"] > 70) &    
                                                           (vacation_weather_df["Wind Speed"] < 10) &                                        
                                                           (vacation_weather_df["Cloudiness"] == 0), columns]

final_vacation_weather_df = intermediary_vacation_weather_df.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [115]:
# Importing json, Storing our dataframe into the variable name hotel_df and searching for hotels within 5000 meters.
import json

hotel_df = final_vacation_weather_df
hotel_df["Hotel Name"] = ""

params = {"radius": 5000,
          "types": "lodging",
          "keyword": "Hotel",
          "key": g_key
          }

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_address = requests.get(base_url, params=params)
    
    hotel_address = hotel_address.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [116]:
# Retreiving our hotels data and plotting a data frame from it. 
hotel_df.drop(hotel_df[hotel_df["Hotel Name"] == ""].index, inplace = True)
hotel_df = hotel_df.head(10)
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Hotel Name"]]
hotel_df

,City,Country,Lat,Lng,Hotel Name
91,Ponta do Sol,PT,32.67,-17.10,Palace of Zinos
128,Jales,BR,-20.27,-50.55,Regional Plaza Hotel
177,Ribas do Rio Pardo,BR,-20.44,-53.76,Maqueza Park Hotel
194,Beyneu,KZ,45.32,55.20,"HOTEL ""AKNUR"""
214,Naviraí,BR,-23.07,-54.19,Hotel Villa Verde
224,Huilong,CN,31.81,121.66,Hilton Garden Inn Qidong
320,Mizdah,LY,31.45,12.98,فندق مزدة الكبير
341,Tūkrah,LY,32.53,20.58,قاعدة فيقا فيقا
357,Ilhabela,BR,-23.78,-45.36,Ilha Flat Hotel
472,Mahon,ES,39.89,4.27,Casa Ládico Boutique Hotel


In [117]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Storing the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [118]:
# Adding a marker layer at the top of the heat map and displaying our gmap figure. 
figure_layout = {"width": "900px",
                 "height": "600px",
                 "border": "1px solid black",
                 "padding": "1px",
                 "margin": "0 auto 0 auto"
                 }

my_gmap = gmaps.figure(layout=figure_layout, zoom_level=2, center=(15,25))

hotel_layer_mark = gmaps.marker_layer(locations, info_box_content=hotel_info)
heat_structure = gmaps.heatmap_layer(lat_lng_loc, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3.5)

my_gmap.add_layer(heat_structure)
my_gmap.add_layer(hotel_layer_mark)
my_gmap.add_layer(hotel_layer_mark)

my_gmap

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…